# Analyzing variable importance using logistic regression
### Christian Igel, 2021-2024

This notebook demonstrates how to analyze variable importance using logistic regression. It reimplements the example in R from [https://stats.idre.ucla.edu/r/dae/logit-regression](https://stats.idre.ucla.edu/r/dae/logit-regression) in Python. It is reassuring to reproduce the R results. 

The notebook demonstrates how to do the analysis of the variable importance using the logistic regression [statmodels](https://www.statsmodels.org/). Note that we do not use scikit-learn here.
First, we will do the data preprocessing using [pandas](https://pandas.pydata.org/).
Second, the notebook will show an alternative version that uses [patsy](https://patsy.readthedocs.io).
Accordingly, you may need to install [pandas](https://pandas.pydata.org/), [statmodels](https://www.statsmodels.org/), and [patsy](https://patsy.readthedocs.io).

Any suggestions for improvements of this notebook are more than welcome.

In [1]:
import pandas as pd
import statsmodels.api as sm

The task is to predict admission into graduate school based on GRE (Graduate Record Exam scores), GPA (grade point average) and prestige of the undergraduate institution. Let's load the data:

In [2]:
df = pd.read_csv("https://stats.idre.ucla.edu/stat/data/binary.csv")

Let's inspect the data:

In [3]:
df.head(5)

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [4]:
df.describe()

,admit,gre,gpa,rank
count,400.000000,400.000000,400.000000,400.00000
mean,0.317500,587.700000,3.389900,2.48500
std,0.466087,115.516536,0.380567,0.94446
min,0.000000,220.000000,2.260000,1.00000
25%,0.000000,520.000000,3.130000,2.00000
50%,0.000000,580.000000,3.395000,2.00000
75%,1.000000,660.000000,3.670000,3.00000
max,1.000000,800.000000,4.000000,4.00000


### First version
In the first version, we do the main data preprocessing steps ourselves.

The goal is to predict/explain the binary variable `admit` given the other variables. Thus, we split data into input (predictor variables) and target (response/dependent variable):

In [5]:
X = df.iloc[:,1:4]
y = df.iloc[:,0]

We treat the `rank`, which indicates the prestige of the undergraduate institution, as a categorical variable:

In [6]:
X["rank"] = X["rank"].astype('category')

Let's look at the data again:

In [7]:
display(X)
display(y)

,gre,gpa,rank
0,380,3.61,3
1,660,3.67,3
2,800,4.00,1
3,640,3.19,4
4,520,2.93,4
...,...,...,...
395,620,4.00,2
396,560,3.04,3
397,460,2.63,2
398,700,3.65,2


0      0
1      1
2      1
3      1
4      0
      ..
395    0
396    0
397    0
398    0
399    0
Name: admit, Length: 400, dtype: int64

Now we transform the categorical variable. Note that in contrast to a one-hot-encoding, one column is dropped to avoid the linear dependency. This means, we use three columns to encode the four ranks. **rank_1**  is encoded by all three columns being zero.

In [8]:
X_transformed = pd.get_dummies(X, prefix=['rank'], drop_first=True)
X_transformed

,gre,gpa,rank_2,rank_3,rank_4
0,380,3.61,False,True,False
1,660,3.67,False,True,False
2,800,4.00,False,False,False
3,640,3.19,False,False,True
4,520,2.93,False,False,True
...,...,...,...,...,...
395,620,4.00,True,False,False
396,560,3.04,False,True,False
397,460,2.63,True,False,False
398,700,3.65,True,False,False


The logistic regression model we are using does not have a built in intercept (bias/offset) parameter. Thus, we augment our input data with a constant dummy variable. 

In [9]:
X_transformed = sm.add_constant(X_transformed)
X_transformed

,const,gre,gpa,rank_2,rank_3,rank_4
0,1.0,380,3.61,False,True,False
1,1.0,660,3.67,False,True,False
2,1.0,800,4.00,False,False,False
3,1.0,640,3.19,False,False,True
4,1.0,520,2.93,False,False,True
...,...,...,...,...,...,...
395,1.0,620,4.00,True,False,False
396,1.0,560,3.04,False,True,False
397,1.0,460,2.63,True,False,False
398,1.0,700,3.65,True,False,False


Now we can compute and inspect the logistic regression model: 

In [10]:
logit_model=sm.Logit(y, X_transformed.astype(float))
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.573147
         Iterations 6
                         Results: Logit
Model:              Logit            Method:           MLE       
Dependent Variable: admit            Pseudo R-squared: 0.083     
Date:               2024-07-07 18:03 AIC:              470.5175  
No. Observations:   400              BIC:              494.4663  
Df Model:           5                Log-Likelihood:   -229.26   
Df Residuals:       394              LL-Null:          -249.99   
Converged:          1.0000           LLR p-value:      7.5782e-08
No. Iterations:     6.0000           Scale:            1.0000    
-------------------------------------------------------------------
           Coef.    Std.Err.      z      P>|z|     [0.025    0.975]
-------------------------------------------------------------------
const     -3.9900     1.1400   -3.5001   0.0005   -6.2242   -1.7557
gre        0.0023     0.0011    2.0699   0.0385    0.0001 

### Second version

In the second version, we use a library function for creating design matrices:

In [11]:
from patsy import dmatrices

We can use a formula syntax to specify the design matrix as in R. When a formula is used to specify the terms to include in the design matrix, a constant for the intercept term will be included by default. The function `C` handles the encoding of the categorical variable for us:

In [12]:
y, X = dmatrices('admit ~ gre + gpa + C(rank)', df, return_type = 'dataframe')
X.head()

,Intercept,C(rank)[T.2],C(rank)[T.3],C(rank)[T.4],gre,gpa
0,1.0,0.0,1.0,0.0,380.0,3.61
1,1.0,0.0,1.0,0.0,660.0,3.67
2,1.0,0.0,0.0,0.0,800.0,4.00
3,1.0,0.0,0.0,1.0,640.0,3.19
4,1.0,0.0,0.0,1.0,520.0,2.93


In [13]:
logit = sm.Logit(y, X)
result = logit.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.573147
         Iterations 6
                         Results: Logit
Model:              Logit            Method:           MLE       
Dependent Variable: admit            Pseudo R-squared: 0.083     
Date:               2024-07-07 18:03 AIC:              470.5175  
No. Observations:   400              BIC:              494.4663  
Df Model:           5                Log-Likelihood:   -229.26   
Df Residuals:       394              LL-Null:          -249.99   
Converged:          1.0000           LLR p-value:      7.5782e-08
No. Iterations:     6.0000           Scale:            1.0000    
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
Intercept     -3.9900    1.1400  -3.5001  0.0005  -6.2242  -1.7557
C(rank)[T.2]  -0.6754    0.3165  -2.1342  0.0328  -1.2958  -0.

### Interpreting the resuls
The results table  shows the coefficients, their standard errors, the z-statistic (sometimes called a Wald z-statistic, in this case the z-value is computed as the coefficient over its standard error, a concept we have not discussed in the lecture), the associated p-values (P), as well as the confidence intervals for the coefficient estimates. 

Assuming a 5% significance level, one can argue that all predictor variables are statistically significant, because their p-values are all smaller than 0.05. (The null hypothesis is that the coefficient is zero. If the p-value is smaller than our significance level, we reject the null hypothesis.) 

"The logistic regression coefficients give the change in the log odds of the outcome for a one unit increase in the predictor variable. For every one unit change in `gre`, the log odds of admission (versus non-admission) increases by 0.002. For a one unit increase in `gpa`, the log odds of being admitted to graduate school increases by 0.804. The indicator variables for `rank` have a slightly different interpretation. For example, having attended an undergraduate institution with `rank` of 2, versus an institution with a `rank` of 1, changes the log odds of admission by -0.675" [(quoted from here)](https://stats.idre.ucla.edu/r/dae/logit-regression/).

The accuracy of the model on the training set, which is *not* an ubiased estimate of the generalizaition performance, can be computed as follows:

In [14]:
from sklearn.metrics import accuracy_score
y_pred = result.predict(X)  # Predict using the result object
print('accuracy on training set:', 
      accuracy_score(y, [1 if m >= 0.5 else 0 for m in y_pred]))

accuracy on training set: 0.71


### Advanced considerations$^*$
#### Standard errors (and Fisher information)
We find the coefficients via maximum likelihood estimation (MLE).
Assume that the data are indeed generated by a model that can represented by a logistic regression model with optimal parameters $\mathbf{w}^*$.
Let the log-likelihood function given data $S=\{(\textbf{x}_1,y_1), \dots, (\textbf{x}_N,y_N)\}$ be denoted by $l_S(\textbf{w})$.
A main theoretical property of MLE is that the estimated parameters $\hat{\mathbf{w}}$ are approximately normally distributed around $\mathbf{w}^*$ with 
covariance matrix equal to the inverse of the *Fisher information matrix* $\mathcal{I}_{\mathbf{w}^*}$:
$$
\hat{\mathbf{w}}\sim\mathcal{N}(\mathbf{w}^*, \mathcal{I}_{\mathbf{w}^*}^{-1})
$$
For $\mathcal{I}_{\mathbf{w}^*}$ we plug in the *observed Fisher information matrix* $\mathcal{J}_{\hat{\mathbf{w}}} = \left.-\nabla_{\textbf{w}} \nabla_{\textbf{w}}^{\text{T}} l_S(\textbf{w})\right|_{\hat{\textbf{w}}}$ with entries
$$
[\mathcal{J}_{\hat{\mathbf{w}}}]_{ij}=\left.-\frac{\partial^2}{\partial w_i \partial w_j} l_S(\textbf{w})\right|_{\hat{\textbf{w}}}.
$$
We already know  the gradient of the negative log-likelihood for logistic regression
$
  %\frac{\partial}{\partial \textbf{w}} 
  -\nabla_{\textbf{w}} l_S(\textbf{w}) = - \sum_{n=1}^N\left[y_n -  \theta(\textbf{w}^\text{T}\textbf{x}_n)\right]\textbf{x}_n
$, which leads to
$$
-\nabla_{\textbf{w}} \nabla_{\textbf{w}}^{\text{T}} l_S(\textbf{w}) = \mathbf{X}^{\text{T}}\mathbf{V}\mathbf{X},
$$
where 
$$
\mathbf{V}=\operatorname{diag}(\theta(\textbf{w}^\text{T}\textbf{x}_1)(1-\theta(\textbf{w}^\text{T}\textbf{x}_1)), \dots,    \theta(\textbf{w}^\text{T}\textbf{x}_N)(1-\theta(\textbf{w}^\text{T}\textbf{x}_N)) )
= \operatorname{diag}(\theta(y_1\textbf{w}^\text{T}\textbf{x}_1)(1-\theta(y_1\textbf{w}^\text{T}\textbf{x}_1)), \dots,    \theta(y_N\textbf{w}^\text{T}\textbf{x}_N)(1-\theta(y_N\textbf{w}^\text{T}\textbf{x}_N)) ).
$$
The *standard error* (SE) of an estimate of a parameter is (an estimate of) the standard deviation of its sampling distribution.
In our case, the SEs are  the square roots of the diagonal entries of the inverse  Fisher matrix, e.g., $\text{SE}(\hat{w}_i)=\sqrt{[\mathcal{J}_{\hat{\mathbf{w}}}^{-1}]_{ii}}$.

Thus, we get the the SEs by hand like this:

In [21]:
import numpy as np
np.set_printoptions(precision=4,suppress=True)
# Compute observed Fisher information matrix
V = np.diag(y_pred * (1. - y_pred))
I = X.to_numpy().T @ V @ X.to_numpy()
# Compute standard errors
I_inv = np.linalg.inv(I)  # inverse Fisher matrix
se = np.sqrt(np.diag(I_inv))
print("standard errors:\n", se)

standard errors:
 [1.14   0.3165 0.3453 0.4178 0.0011 0.3318]


#### Wald statistic and Z-score
The Wald test compares the MLE estimate $\hat{w_i}$ of a parameter 
with a hypothesized value ${w_0}$.
The Z-score is computed as 
$$
z=\frac{\hat{w}_i - w_0}{\text{SE}(\hat{w}_i)}
$$
and asymptotically follows a standard normal distribution.
The larger $|z|$, the more likely is the true ${w}_i^*$ estimated by $\hat{w}_i$ different from ${w}_0$.
The definition of the Z-score is rather intuitive. The absolute value of the numerator increases with the distance between $\hat{w}_i$ and  ${w}_0$. This distance is scaled by the denominator. If the SE (the standard deviation of our estimate) is low, we are very sure about the estimation, and a small distance between $\hat{w}_i$ and  ${w}_0$ is already significant. If the SE is large, we are rather uncertain about our estimate $\hat{w}_i$, and the distance between $\hat{w}_i$ and  ${w}_0$ has to be larger to be significant.

We consider $w_0=0$ and can compute the $z$ values by hand:

In [22]:
w = result.params.to_numpy()  # get the coefficients of logistic regression w/o linear rescaling
z = w/se
print(" z : ", z)
print("|z|: ", abs(z))

 z :  [-3.5001 -2.1342 -3.8812 -3.7131  2.0699  2.4231]
|z|:  [3.5001 2.1342 3.8812 3.7131 2.0699 2.4231]


Now we can look up the  $z$ value for a given confidence level. For a confidence level of 95%, it is 1.96.
Thus, we compare $|z|$ to 1.96

For example, we cannot reject the null hypothesis that the intercept parameter is zero at a confidence level of 95%, because the corresponding $|z|$ value is smaller than 1.95 

#### Confidence intervals for the parameters
We get the confidence intervals for the coefficient estimates by solving for $w_0$ given the $z$ value in question:

In [23]:
z95 = 1.96  # confidence level 95%
print("CI:\n", np.stack( (w - z95 * se, w + z95 * se), axis=-1))

CI:
 [[-6.2243 -1.7557]
 [-1.2958 -0.0551]
 [-2.017  -0.6634]
 [-2.3704 -0.7325]
 [ 0.0001  0.0044]
 [ 0.1537  1.4544]]


### Effect of linear rescaling 
In the following, it is demonstrated that linear rescaling does not change the significance of the the predictor variables.

In [24]:
from sklearn.preprocessing import StandardScaler
X_norm = X.copy()
cols_to_norm = ['gre','gpa']
X_norm[cols_to_norm] = StandardScaler().fit_transform(X_norm[cols_to_norm])

In [25]:
X_norm.head()

,Intercept,C(rank)[T.2],C(rank)[T.3],C(rank)[T.4],gre,gpa
0,1.0,0.0,1.0,0.0,-1.800263,0.579072
1,1.0,0.0,1.0,0.0,0.626668,0.736929
2,1.0,0.0,0.0,0.0,1.840134,1.605143
3,1.0,0.0,0.0,1.0,0.453316,-0.525927
4,1.0,0.0,0.0,1.0,-0.586797,-1.209974


In [26]:
logit = sm.Logit(y, X_norm)
result_norm = logit.fit()
print(result_norm.summary2())

Optimization terminated successfully.
         Current function value: 0.573147
         Iterations 6
                         Results: Logit
Model:              Logit            Method:           MLE       
Dependent Variable: admit            Pseudo R-squared: 0.083     
Date:               2024-07-07 18:04 AIC:              470.5175  
No. Observations:   400              BIC:              494.4663  
Df Model:           5                Log-Likelihood:   -229.26   
Df Residuals:       394              LL-Null:          -249.99   
Converged:          1.0000           LLR p-value:      7.5782e-08
No. Iterations:     6.0000           Scale:            1.0000    
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
Intercept      0.0664    0.2656   0.2502  0.8025  -0.4540   0.5869
C(rank)[T.2]  -0.6754    0.3165  -2.1342  0.0328  -1.2958  -0.

Note that the p-values of the predictor variables did not change, as it should be. 
However, now we  cannot reject the null hypothesis that the intercept parameter is zero, zero is also in the confidence interval of the intercept estimate.